In [1]:
%pip install --upgrade sliderule

Note: you may need to restart the kernel to use updated packages.


In [20]:
%%time

import os
import geopandas as gpd
import pandas as pd
from sliderule import icesat2, earthdata, sliderule
import json

CPU times: user 14 µs, sys: 0 ns, total: 14 µs
Wall time: 17.2 µs


In [31]:
# # Initialize sliderule (ensure this is connected to Earthdata or your desired endpoint)
# sliderule.init("slideruleearth.io")

# Load geojson file for the region of interest (BONA example)
geojson_file = "geojson_files/BONA_buffer_8km.geojson"

# Read in the polygon coordinates using sliderule.toregion
region = sliderule.toregion(geojson_file)

# Ensure the polygon is in the correct format for earthdata.cmr
polygon_coordinates = [{"lat": coord['lat'], "lon": coord['lon']} for coord in region['poly']]

print(polygon_coordinates)
# Set your parameters for time and version
time_start = "2023-01-01T00:00:00Z"
time_end = "2023-12-31T23:59:59Z"
release = "001"

# Query Earthdata's CMR for the granules using the extracted polygon coordinates
granules_list = earthdata.cmr(short_name='ATL03', polygon=polygon_coordinates, 
                               time_start=time_start, time_end=time_end, 
                               version=release)

# Print or process the granules list
print(granules_list)


[{'lat': 65.11812384063202, 'lon': -147.58788094933342}, {'lat': 65.11812384063202, 'lon': -147.41731905066658}, {'lat': 65.18987598639683, 'lon': -147.41731905066658}, {'lat': 65.18987598639683, 'lon': -147.58788094933342}, {'lat': 65.11812384063202, 'lon': -147.58788094933342}]
[]


In [3]:
%%time
geojson_directory = "geojson_files"
years_to_check = list(range(2018, 2024))  # Check years 2018 to 2023

CPU times: user 4 µs, sys: 1 µs, total: 5 µs
Wall time: 8.11 µs


In [11]:
def find_atl03_availability(geojson_dir, years_to_check, product="ATL03", release="003"):
    """
    Find ATL03 data availability by year for a set of sites defined by GeoJSON files.

    Parameters:
    - geojson_dir (str): Directory containing GeoJSON files for site boundaries.
    - years_to_check (list): List of years to check (e.g., from 2018 to 2023).
    - product (str): ICESat-2 product (default: "ATL03").
    - release (str): The release version of the product (default: "003").

    Returns:
    - dict: Dictionary where keys are site names and values are lists of available years.
    """
    # Dictionary to store results
    availability = {}

    # Loop through GeoJSON files in the directory
    for geojson_file in os.listdir(geojson_dir):
        if geojson_file.endswith(".geojson"):
            site_name = os.path.splitext(geojson_file)[0]

            # Use sliderule to read the GeoJSON file and get the region
            region = sliderule.toregion(os.path.join(geojson_dir, geojson_file))

            # Query CMR for the specified product for each year in the specified range
            available_years = []
            for year in years_to_check:
                time_start = f"{year}-01-01T00:00:00Z"
                time_end = f"{year}-12-31T23:59:59Z"

                try:
                    # Query CMR for the granules in the given time range and polygon
                    granules = earthdata.cmr(short_name=product, 
                                              polygon=region, 
                                              time_start=time_start, 
                                              time_end=time_end, 
                                              version=release)
                    
                    # If granules are found, the data is available for the year
                    if granules:
                        available_years.append(year)
                
                except Exception as e:
                    print(f"Error querying CMR for {site_name} in {year}: {e}")

            # Save results for the site
            if available_years:
                availability[site_name] = available_years
    
    return availability

In [12]:
%%time
availability = find_atl03_availability(geojson_directory, years_to_check)
print(availability)

Error querying CMR for DEJU_buffer_8km in 2018: 0
Error querying CMR for DEJU_buffer_8km in 2019: 0
Error querying CMR for DEJU_buffer_8km in 2020: 0
Error querying CMR for DEJU_buffer_8km in 2021: 0
Error querying CMR for DEJU_buffer_8km in 2022: 0
Error querying CMR for DEJU_buffer_8km in 2023: 0
Error querying CMR for BONA_buffer_8km in 2018: 0
Error querying CMR for BONA_buffer_8km in 2019: 0
Error querying CMR for BONA_buffer_8km in 2020: 0
Error querying CMR for BONA_buffer_8km in 2021: 0
Error querying CMR for BONA_buffer_8km in 2022: 0
Error querying CMR for BONA_buffer_8km in 2023: 0
Error querying CMR for WREF_buffer_8km in 2018: 0
Error querying CMR for WREF_buffer_8km in 2019: 0
Error querying CMR for WREF_buffer_8km in 2020: 0
Error querying CMR for WREF_buffer_8km in 2021: 0
Error querying CMR for WREF_buffer_8km in 2022: 0
Error querying CMR for WREF_buffer_8km in 2023: 0
{}
CPU times: user 20.8 ms, sys: 16.4 ms, total: 37.1 ms
Wall time: 42.2 ms
